<a href="https://colab.research.google.com/github/zainkhan-afk/scrabble/blob/master/wallace4x4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import numpy as np
import torch
import torchvision
import matplotlib.pyplot as plt
from time import time
from torchvision import datasets, transforms
from torch import nn, optim
import os

In [0]:
transform = transforms.Compose([transforms.ToTensor(),
                              ])

transformTest = transforms.Compose([transforms.ToTensor(),
                              ])

In [0]:
X = np.zeros([16, 1])
multiplicand = np.zeros([8, 1])
multiplier = np.zeros([8, 1])
for i in range(0, 2):
  for j in range(0, 2):
    bin = format(i, "b")
data = multiplicand
data = np.append(data, multiplier, 0)
X = np.append(X, data, 1)
print(multiplier.shape)
print(data.shape)
print(X.shape)

(8, 1)
(16, 1)
(16, 2)


**Loading the Training Data**

In [0]:
Data = np.loadtxt('NoisyXORTrainingData.txt')
Y = np.zeros([8, 1])
Data = Data[:, :8]
print(Data.shape)

for d in range(0, 5000):
  multiplicand = 0
  multiplier = 0

  for i in range(0, 4):
    multiplicand = multiplicand + Data[d, i]*2**i
    multiplier = multiplier +  Data[d, i+4]*2**i

  result = multiplicand*multiplier
  resultBin = format(int(result), "b")
  opLen = len(resultBin)
  op = []
  for i in range(0, 8):
    if i<opLen:
      op.append(int(resultBin[i]))
    else:
      op.append(0)
  op = np.array(op)
  op = op[:, np.newaxis]
  Y = np.append(Y, op, 1)

Y = np.transpose(Y)
Y = Y[1:, :]
print(Y.shape)
print(Y)

(5000, 8)
(5000, 8)
[[1. 1. 0. ... 0. 0. 0.]
 [1. 1. 0. ... 1. 0. 0.]
 [1. 1. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [1. 1. 0. ... 0. 0. 0.]
 [1. 0. 0. ... 0. 0. 0.]]


**Loading the Testing Data**

In [0]:
DataTest = np.loadtxt('NoisyXORTestData.txt')
YTest = np.zeros([8, 1])
DataTest = DataTest[:, :8]
print(DataTest.shape)

for d in range(0, 5000):
  multiplicand = 0
  multiplier = 0

  for i in range(0, 4):
    multiplicand = multiplicand + DataTest[d, i]*2**i
    multiplier = multiplier +  DataTest[d, i+4]*2**i

  result = multiplicand*multiplier
  resultBin = format(int(result), "b")
  opLen = len(resultBin)
  op = []
  for i in range(0, 8):
    if i<opLen:
      op.append(int(resultBin[i]))
    else:
      op.append(0)
  op = np.array(op)
  op = op[:, np.newaxis]
  YTest = np.append(YTest, op, 1)

YTest = np.transpose(YTest)
YTest = YTest[1:, :]
print(YTest.shape)
print(YTest)

(5000, 8)
(5000, 8)
[[1. 0. 0. ... 0. 0. 0.]
 [1. 1. 0. ... 0. 0. 0.]
 [1. 0. 0. ... 0. 0. 0.]
 ...
 [1. 0. 0. ... 0. 0. 0.]
 [1. 0. 0. ... 0. 0. 0.]
 [1. 0. 1. ... 1. 1. 0.]]


In [0]:
trainloader = torch.utils.data.DataLoader(Data, batch_size=64, shuffle=False)
labels = torch.utils.data.DataLoader(Y, batch_size=64, shuffle=False)

testloader = torch.utils.data.DataLoader(DataTest, batch_size=64, shuffle=False)
labelsTest = torch.utils.data.DataLoader(YTest, batch_size=64, shuffle=False)

# dataiter = iter(trainloader)
# print(dataiter.shape)
# images= dataiter.next()
# print(type(images))
# print(images.shape)

# Yiter = iter(labels)
# labs = Yiter.next()
# print(labs.shape)

In [0]:
input_size = 8
hidden_sizes = [128, 64]
output_size = 8

model = nn.Sequential(nn.Linear(input_size, hidden_sizes[0]),
                      nn.ReLU(),
                      nn.Linear(hidden_sizes[0], hidden_sizes[1]),
                      nn.ReLU(),
                      nn.Linear(hidden_sizes[1], hidden_sizes[0]),
                      nn.ReLU(),
                      nn.Linear(hidden_sizes[0], hidden_sizes[1]),
                      nn.ReLU(),
                      nn.Linear(hidden_sizes[1], output_size),
                      nn.Sigmoid())
print(model)

Sequential(
  (0): Linear(in_features=8, out_features=128, bias=True)
  (1): ReLU()
  (2): Linear(in_features=128, out_features=64, bias=True)
  (3): ReLU()
  (4): Linear(in_features=64, out_features=128, bias=True)
  (5): ReLU()
  (6): Linear(in_features=128, out_features=64, bias=True)
  (7): ReLU()
  (8): Linear(in_features=64, out_features=8, bias=True)
  (9): Sigmoid()
)


In [0]:
criterion = nn.MSELoss()
optimizer = optim.SGD(model.parameters(), lr=0.05, momentum=0.9)
time0 = time()
epochs = 200

In [0]:
for e in range(epochs):
    running_loss = 0
    Yiter = iter(labels)
    for images in trainloader:
        # Flatten MNIST images into a 784 long vector
        # images = images.view(images.shape[0], -1)
        labs = Yiter.next()
        # Training pass
        optimizer.zero_grad()
        
        # print("IP: ", images.shape)
        output = model(images.float())
        # print("OP: ", output.shape)
        # print("Truth: ", labs.shape)
        loss = criterion(output, labs.float())
        
        #This is where the model learns by backpropagating
        loss.backward()
        
        #And optimizes its weights here
        optimizer.step()
        
        running_loss += loss.item()
    # else:
    print("Epoch {} - Training loss: {}".format(e, running_loss/len(trainloader)))
print("\nTraining Time (in minutes) =",(time()-time0)/60)

Epoch 0 - Training loss: 0.2062606092872499
Epoch 1 - Training loss: 0.16672549085526528
Epoch 2 - Training loss: 0.1651590130751646
Epoch 3 - Training loss: 0.16452400612680218
Epoch 4 - Training loss: 0.16372081505346903
Epoch 5 - Training loss: 0.16234105240695085
Epoch 6 - Training loss: 0.15945697245718557
Epoch 7 - Training loss: 0.1530823656652547
Epoch 8 - Training loss: 0.14494312149059924
Epoch 9 - Training loss: 0.13967094919349574
Epoch 10 - Training loss: 0.13327113359789305
Epoch 11 - Training loss: 0.12644114741418935
Epoch 12 - Training loss: 0.11996359849655175
Epoch 13 - Training loss: 0.11368234983727901
Epoch 14 - Training loss: 0.10810346920279008
Epoch 15 - Training loss: 0.10325329703620717
Epoch 16 - Training loss: 0.09860707347906089
Epoch 17 - Training loss: 0.09390196279634404
Epoch 18 - Training loss: 0.08921563568749005
Epoch 19 - Training loss: 0.08495094976093195
Epoch 20 - Training loss: 0.0814338847051693
Epoch 21 - Training loss: 0.07861353362662883
Ep

In [0]:
correct_count, all_count = 0, 0
Yiter = iter(labelsTest)
for images in testloader:
  output = model(images.float())
  output = output.round() 
  # print(output.round())
  labs = Yiter.next()
  # print("OP: ", output.shape)
  # print("TRUTH: ", labs.shape)
  R, C = output.shape
  for r in range(0, R):
    res = abs(output[r, :] - labs[r, :])
    res = sum(res)
    if res == 0:
      correct_count += 1
    all_count += 1
  # for i in range(len(labels)):
  #   img = images[i].view(1, 784)
  #   with torch.no_grad():
  #       logps = model(img)

    
  #   ps = torch.exp(logps)
  #   probab = list(ps.numpy()[0])
  #   pred_label = probab.index(max(probab))
  #   true_label = labels.numpy()[i]
  #   if(true_label == pred_label):
  #     correct_count += 1
  #   all_count += 1

print("Number Of Samples Tested =", all_count)
print("\nModel Accuracy =", (correct_count/all_count))

Number Of Samples Tested = 5000

Model Accuracy = 0.9032
